In [16]:
import pandas as pd
import math
import datetime
from datetime import datetime, timedelta
from datetime import datetime, time
import os
# ids = str(199)
file_path_acc = os.path.join('/home/ubuntu/eSportData/jupyter-data/akcelerometry/S1/S1_245.csv')
txs = ['245_m0.txt','245_m1.txt', '245_m2.txt','245_m3.txt','245_m4.txt']
# time = '11:50:10.500'
df_dz = pd.read_csv("/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1/245_08111644_DZ.txt", sep=" ", header=None, names=['index', 'DZ', 'marker1', 'marker2', 'dummy', 'dummy2'], skiprows=3)
df_dzdt = pd.read_csv("/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1/245_08111644_DZDT.txt", sep=" ", header=None, names=['index', 'DZDT', 'marker1', 'marker2', 'dummy', 'dummy2'], skiprows=3)
df_ecg = pd.read_csv("/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1/245_08111644_ECG.txt", sep=" ", header=None, names=['index', 'ECG', 'marker1', 'marker2', 'dummy', 'dummy2'], skiprows=3)
df_z0 = pd.read_csv("/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1/245_08111644_Z0.txt", sep=" ", header=None, names=['index', 'Z0', 'marker1', 'marker2', 'dummy', 'dummy2'], skiprows=3)
df_z0 = pd.concat([pd.DataFrame({'index': [df_z0['index'].min() - 1], 'Z0': [df_z0['Z0'].iloc[0]], 'marker1': [df_z0['marker1'].iloc[0]]}), df_z0])
new_index = range(df_z0['index'].min(), df_z0['index'].max() + 1)
df_z0 = df_z0.set_index('index').reindex(new_index)                    
df_z0 = df_z0.ffill()
df_z0 = df_z0.reset_index()

In [17]:
col_names = ['timestamp','ECG', 'DZ', 'DZDT', 'Z0', 'marker']
markers_keys = {'1':'m0', '15':'m1', '21':'m2', '25': 'm3', '35':'m4'}

In [19]:
def extract_time_from_file(filename):
    try:
        with open(filename, 'r') as file:
            # Skip the first row
            next(file)
            # Read the second row
            second_row = next(file).strip()
            # Split by "/"
            parts = second_row.split("/")
            # Return the time part after "/"
            return parts[1]
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return None

In [20]:
time = extract_time_from_file("/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1/245_08111644_DZ.txt")

In [21]:
def calculate_deltatime(time_col_vms, time_col_bs):
    time_diff = datetime.strptime(str(time_col_vms), '%H:%M:%S.%f') - datetime.strptime(str(time_col_bs),'%H:%M:%S.%f')
    return time_diff

In [22]:
ecg_col = df_ecg['ECG']
dz_col = df_dz['DZ']
dzdt_col = df_dzdt['DZDT']
z0_col = df_z0['Z0']
marker = df_ecg['marker2']
min_length = len(df_z0)
df_ecg = df_ecg.head(min_length)
df_dz = df_dz.head(min_length)
df_dzdt = df_dzdt.head(min_length)
data_dict = {'ECG': ecg_col, 'DZ': dz_col, 'DZDT': dzdt_col, 'Z0': z0_col, 'marker':marker}
da = pd.DataFrame(data_dict, index=df_dzdt.index, columns=col_names)

In [23]:
da

,timestamp,ECG,DZ,DZDT,Z0,marker
0,NaN,0.192009,0.817136,0.007630,14.809035,-9999
1,NaN,0.187432,0.817136,0.083925,14.809035,-9999
2,NaN,0.182684,0.817288,0.190738,14.809035,-9999
3,NaN,0.177429,0.817502,0.175479,14.809035,-9999
4,NaN,0.173020,0.817624,0.175479,14.809035,-9999
...,...,...,...,...,...,...
6508973,NaN,5.484347,0.989990,0.022889,26.265934,-9999
6508974,NaN,5.484347,0.989990,0.068666,26.265934,-9999
6508975,NaN,5.484008,0.990112,0.129702,26.265934,-9999
6508976,NaN,5.484347,0.990234,0.038148,26.265934,-9999


In [28]:
da.iloc[0,0] = time

In [29]:
da

,timestamp,ECG,DZ,DZDT,Z0,marker
0,16:44:03.500,0.192009,0.817136,0.007630,14.809035,-9999
1,NaN,0.187432,0.817136,0.083925,14.809035,-9999
2,NaN,0.182684,0.817288,0.190738,14.809035,-9999
3,NaN,0.177429,0.817502,0.175479,14.809035,-9999
4,NaN,0.173020,0.817624,0.175479,14.809035,-9999
...,...,...,...,...,...,...
6508973,NaN,5.484347,0.989990,0.022889,26.265934,-9999
6508974,NaN,5.484347,0.989990,0.068666,26.265934,-9999
6508975,NaN,5.484008,0.990112,0.129702,26.265934,-9999
6508976,NaN,5.484347,0.990234,0.038148,26.265934,-9999


In [30]:
print(da.index[da['marker'] == 1])

Index([1065695, 1065696, 1065697, 1065698, 1065699, 1065700, 1065701, 1065702,
       1065703, 1065704,
       ...
       1125685, 1125686, 1125687, 1125688, 1125689, 1125690, 1125691, 1125692,
       1125693, 1125694],
      dtype='int64', length=60000)


In [31]:
print(da.iloc[0])

timestamp    16:44:03.500
ECG              0.192009
DZ               0.817136
DZDT              0.00763
Z0              14.809035
marker              -9999
Name: 0, dtype: object


In [32]:
da['timestamp'] = pd.to_datetime(da['timestamp'], format='%H:%M:%S.%f')
first_valid_index = 0

not_null_time = da.loc[first_valid_index, 'timestamp']
da_list = da.values.tolist()
for i in range(first_valid_index + 1, len(da_list)):
    not_null_time += timedelta(milliseconds=1)
    # time_values_str = [time.strftime('%H:%M:%S.%f') for time in time_values]
    da_list[i][0] = not_null_time
da = pd.DataFrame(da_list, columns =col_names)
da['timestamp'] = da['timestamp'].dt.time

In [33]:
da

,timestamp,ECG,DZ,DZDT,Z0,marker
0,16:44:03.500000,0.192009,0.817136,0.007630,14.809035,-9999
1,16:44:03.501000,0.187432,0.817136,0.083925,14.809035,-9999
2,16:44:03.502000,0.182684,0.817288,0.190738,14.809035,-9999
3,16:44:03.503000,0.177429,0.817502,0.175479,14.809035,-9999
4,16:44:03.504000,0.173020,0.817624,0.175479,14.809035,-9999
...,...,...,...,...,...,...
6508973,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999
6508974,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999
6508975,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999
6508976,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999


In [36]:
import re
import os

bs_columns = ['timestamp', 'SBP', 'DBP', 'MBP', 'HR', 'SV', 'LVET', 'PI', 'MS', 'CO', 'TPR', 'TPRCGS', 'marker_tx', 'dummy']
extended_txs = []
# pattern = fr'(?<![0-9m]){re.escape(ids)}(?![0-9])'

txs = ['245_m0.txt','245_m1.txt','245_m3.txt','245_m4.txt']
txs.sort()
print(txs)
for tx in txs:
    file_path_tx = os.path.join('/home/ubuntu/eSportData/jupyter-data/beatscope/Pliki/S1', tx)
    tx_df = pd.read_csv(file_path_tx, sep=';', header=None, skiprows=10, encoding='cp1250', decimal=',', names=bs_columns)
    # Convert timestamp using pd.DatetimeIndex
    tx_df['timestamp'] = pd.to_datetime(tx_df['timestamp'], format='%H:%M:%S,%f')
    print(tx_df.timestamp[100])
                # Dropping duplicates

    # tx_df = tx_df[~tx_df.duplicated('timestamp', keep='first')]
    # try:
    #     tx_df = tx_df[~tx_df.duplicated('timestamp', keep='first')]
    #     print(tx_df.index[tx_df['marker_tx'].notna()][0])
    # except:
    #     tx_df = tx_df[~tx_df.duplicated('timestamp', keep='last')]
    #     print(tx_df.index[tx_df['marker_tx'].notna()][0])
                # Extending
    tx_df = tx_df.set_index('timestamp')
    tx_df = tx_df.resample('ms').ffill()
    tx_df = tx_df.reset_index()
    tx_df['timestamp'] = tx_df['timestamp'].dt.time
    extended_txs.append(tx_df)

        # Asserting end of bs measurement         
    extended_txs[-1].iloc[extended_txs[-1].index.max(), -2] = 'End'
    

['245_m0.txt', '245_m1.txt', '245_m3.txt', '245_m4.txt']
1900-01-01 16:58:41.621000
1900-01-01 17:12:47.744000
1900-01-01 18:23:14.861000
1900-01-01 18:30:55.893000


In [37]:
# extended_txs[0]

In [38]:
da_sync_idx = da.index[da['marker'] != -9999][0] - 240000
da_sync_idx2 = da.index[da['marker'] != -9999][0]
for i in range(da_sync_idx, da_sync_idx2):
    da.iloc[i, 5] = 1
print('check 3.2')
            
            # da_sync_idx = da.index[da['marker'] != -9999][0] - 240000

garry = extended_txs[0]
tx_sync_idx = garry.index[garry['marker_tx'].notna()][0]

            # print(merged_tx.iloc[tx_sync_idx])
print('check 3.3')
            
assert garry.at[tx_sync_idx, 'marker_tx'].startswith('m')
            
rows_diff = tx_sync_idx - da_sync_idx
print(rows_diff)
if rows_diff > 0:  # 
    garry = garry.iloc[rows_diff:]
    garry = garry.reset_index(drop=True)
else:
    da = da.iloc[-rows_diff:]
    da = da.reset_index(drop=True)

check 3.2
check 3.3
-766869


In [39]:
print(da.iloc[183529])

timestamp    16:59:53.898000
ECG                 0.244738
DZ                  0.063432
DZDT               -0.068666
Z0                 35.870574
marker                     1
Name: 183529, dtype: object


In [40]:
da

,timestamp,ECG,DZ,DZDT,Z0,marker
0,16:56:50.369000,-0.002458,0.068711,-0.007630,35.870574,-9999
1,16:56:50.370000,-0.004323,0.069047,0.038148,35.870574,-9999
2,16:56:50.371000,-0.005171,0.068772,-0.099184,35.870574,-9999
3,16:56:50.372000,-0.005002,0.068833,0.083925,35.870574,-9999
4,16:56:50.373000,-0.003815,0.068925,0.007630,35.870574,-9999
...,...,...,...,...,...,...
5742104,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999
5742105,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999
5742106,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999
5742107,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999


In [41]:
da_bc = da.merge(garry, how='left', left_index=True, right_index=True)
# da_na = pd.DataFrame(columns=bs_columns, index=extended_txs[0].index)
# da_bc = da.merge(da_na, how='left', left_index=True, right_index=True)

In [42]:
da_bc

,timestamp_x,ECG,DZ,DZDT,Z0,marker,timestamp_y,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,marker_tx,dummy
0,16:56:50.369000,-0.002458,0.068711,-0.007630,35.870574,-9999,16:57:41.545000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
1,16:56:50.370000,-0.004323,0.069047,0.038148,35.870574,-9999,16:57:41.546000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
2,16:56:50.371000,-0.005171,0.068772,-0.099184,35.870574,-9999,16:57:41.547000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
3,16:56:50.372000,-0.005002,0.068833,0.083925,35.870574,-9999,16:57:41.548000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
4,16:56:50.373000,-0.003815,0.068925,0.007630,35.870574,-9999,16:57:41.549000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742104,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742105,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742106,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742107,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
print(da_bc.index[da_bc['marker'] == 1])

Index([ 58826,  58827,  58828,  58829,  58830,  58831,  58832,  58833,  58834,
        58835,
       ...
       358816, 358817, 358818, 358819, 358820, 358821, 358822, 358823, 358824,
       358825],
      dtype='int64', length=300000)


In [44]:
print(da_bc.iloc[0])

timestamp_x    16:56:50.369000
ECG                  -0.002458
DZ                    0.068711
DZDT                  -0.00763
Z0                   35.870574
marker                   -9999
timestamp_y    16:57:41.545000
SBP                        0.0
DBP                        0.0
MBP                        0.0
HR                        96.0
SV                         0.0
LVET                       0.0
PI                       625.0
MS                         0.0
CO                         0.0
TPR                        0.0
TPRCGS                     0.0
marker_tx                  NaN
dummy                      NaN
Name: 0, dtype: object


In [45]:
# first_non_nan_index = da_bc.index[da_bc['marker'] != -9999][0]

# da_bc.iloc[first_non_nan_index]

In [46]:
da_bc

,timestamp_x,ECG,DZ,DZDT,Z0,marker,timestamp_y,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,marker_tx,dummy
0,16:56:50.369000,-0.002458,0.068711,-0.007630,35.870574,-9999,16:57:41.545000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
1,16:56:50.370000,-0.004323,0.069047,0.038148,35.870574,-9999,16:57:41.546000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
2,16:56:50.371000,-0.005171,0.068772,-0.099184,35.870574,-9999,16:57:41.547000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
3,16:56:50.372000,-0.005002,0.068833,0.083925,35.870574,-9999,16:57:41.548000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
4,16:56:50.373000,-0.003815,0.068925,0.007630,35.870574,-9999,16:57:41.549000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742104,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742105,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742106,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742107,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
time_col = da['timestamp']
# extended_txs[0]['time'] = time_col
# extended_txs[1]['time'] = time_col
# extended_txs[2]['time'] = time_col
# extended_txs[3]['time'] = time_col

In [48]:
trials = [pd.merge(time_col, tx, on='timestamp', how='left') for tx in extended_txs]

In [49]:
extended_txs[0]

,timestamp,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,marker_tx,dummy
0,16:57:41.545000,0,0,0,96,0.0,0,625,0,0.0,0.000,0,NaN,NaN
1,16:57:41.546000,0,0,0,96,0.0,0,625,0,0.0,0.000,0,NaN,NaN
2,16:57:41.547000,0,0,0,96,0.0,0,625,0,0.0,0.000,0,NaN,NaN
3,16:57:41.548000,0,0,0,96,0.0,0,625,0,0.0,0.000,0,NaN,NaN
4,16:57:41.549000,0,0,0,96,0.0,0,625,0,0.0,0.000,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498165,17:05:59.710000,150,112,130,118,44.8,245,510,733,5.3,1.482,1975,NaN,NaN
498166,17:05:59.711000,150,112,130,118,44.8,245,510,733,5.3,1.482,1975,NaN,NaN
498167,17:05:59.712000,150,112,130,118,44.8,245,510,733,5.3,1.482,1975,NaN,NaN
498168,17:05:59.713000,150,112,130,118,44.8,245,510,733,5.3,1.482,1975,NaN,NaN


In [50]:
trials[0]

,timestamp,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,marker_tx,dummy
0,16:56:50.369000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16:56:50.370000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16:56:50.371000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16:56:50.372000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16:56:50.373000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742104,18:32:32.473000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742105,18:32:32.474000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742106,18:32:32.475000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742107,18:32:32.476000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
print(extended_txs[3].index[extended_txs[3]['marker_tx'] == 'm3'])

Index([], dtype='int64')


In [52]:
da

,timestamp,ECG,DZ,DZDT,Z0,marker
0,16:56:50.369000,-0.002458,0.068711,-0.007630,35.870574,-9999
1,16:56:50.370000,-0.004323,0.069047,0.038148,35.870574,-9999
2,16:56:50.371000,-0.005171,0.068772,-0.099184,35.870574,-9999
3,16:56:50.372000,-0.005002,0.068833,0.083925,35.870574,-9999
4,16:56:50.373000,-0.003815,0.068925,0.007630,35.870574,-9999
...,...,...,...,...,...,...
5742104,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999
5742105,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999
5742106,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999
5742107,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999


In [66]:
txs_list = [trial.values.tolist() for trial in trials]
# txs_list = [trial.values.tolist() for trial in trials[1:]]
# tx_list = txs_list[0]
da_bc_list = da_bc.values.tolist()

In [67]:
print(da_bc.index[da_bc['marker_tx'] == 'm0'])

Index([58826, 58827, 58828, 58829, 58830, 58831, 58832, 58833, 58834, 58835,
       ...
       59431, 59432, 59433, 59434, 59435, 59436, 59437, 59438, 59439, 59440],
      dtype='int64', length=615)


In [68]:
# print(da_bc.iloc[4180265])

In [69]:
len(extended_txs)

4

In [70]:
markers_keys = {'1':'m0', '15':'m1', '21':'m2', '25':'m3', '35':'m4'}
start_0 =  da_bc.index[da_bc['marker'] == 1][0]
start_1 =  da_bc.index[da_bc['marker'] ==15][0]
start_2 = da_bc.index[da_bc['marker'] == 21][0]
start_3 = da_bc.index[da_bc['marker'] == 25][0]
start_4 = da_bc.index[da_bc['marker'] == 35][0]
start_markers = [start_0,start_1, start_2, start_3, start_4]

In [ ]:
start_1

In [ ]:
# da_sync_idx_m1 = da.index[da['marker'] == 15][0]

# # garry = extended_txs[0]
# tx_sync_idx_m1 = extended_txs[1].index[extended_txs[1]['marker_tx'].notna()][0]

#             # print(merged_tx.iloc[tx_sync_idx])
# print('check 3.3')
            
# assert extended_txs[1].at[tx_sync_idx_m1, 'marker_tx'].startswith('m1')
            
# rows_diff = tx_sync_idx_m1 - da_sync_idx_m1
# print(rows_diff)
# # if rows_diff > 0:  # 
# #     garry = garry.iloc[rows_diff:]
# #     garry = garry.reset_index(drop=True)
# # else:
# #     da = da.iloc[-rows_diff:]
# #     da = da.reset_index(drop=True)

In [ ]:
# da_sync_value = str(da.loc[start_markers[0], 'marker'])
# print(da_sync_value)
# which_index = start_markers.index(start_markers[1])
# print(which_index)
# mk = str(15)
# tx_sync_idx_m1 = extended_txs[which_index].index[extended_txs[which_index]['marker_tx'] == markers_keys[da_sync_value]][0]

In [ ]:
da_bc.iloc[start_1]

In [ ]:
print(trials[2].index[trials[2]['marker_tx'] == 'm4'])

In [ ]:
# txs_list[4][-9200]

In [71]:
da_bs_columns = ['timestamp_x', 'ECG','DZ' ,'DZDT','Z0', 'marker', 'timestamp_y', 'SBP', 'DBP', 'MBP', 'HR', 'SV', 'LVET', 'PI', 'MS', 'CO', 'TPR', 'TPRCGS', 'marker_tx','dummy']
# for trial in range(len(txs_list)):  #df
for markers in start_markers[1:]:
    # da_sync_idx_m1 = da.index[da['marker'] == markers][0]
    da_sync_value = str(da.loc[markers, 'marker'])
    print(da_sync_value)
    which_index = start_markers.index(markers)
    print(which_index)
    tx_sync_idx_m1 = trials[which_index].index[trials[which_index]['marker_tx'] == markers_keys[da_sync_value]][0]
    print(trials[which_index].index[trials[which_index]['marker_tx'] == markers_keys[da_sync_value]][0])
    print( tx_sync_idx_m1, markers)
    rows_diff = tx_sync_idx_m1 - markers
    print(rows_diff)
   
    for row in range(markers,len(da_bc_list)): #rząd
        # print(da_bc_list[row])
        for value_idx in range(len(da_bc_list[row])):
                if da_bc_list[markers][value_idx] == 15:
                    # print(row, 'row', rows_diff, 'row_diff')
                    # for i_1 in range(row, len(da_bc_list)):
                    #     print(da_bc_list[i_1][1])
                   
                    try:
                        da_bc_list[row][7] = txs_list[1][row+rows_diff][1]
                        da_bc_list[row][8] = txs_list[1][row+rows_diff][2]
                        da_bc_list[row][9] = txs_list[1][row+rows_diff][3]
                        da_bc_list[row][10] = txs_list[1][row+rows_diff][4]
                        da_bc_list[row][11] = txs_list[1][row+rows_diff][5]
                        da_bc_list[row][12] = txs_list[1][row+rows_diff][6]
                        da_bc_list[row][13] = txs_list[1][row+rows_diff][7]
                        da_bc_list[row][14] = txs_list[1][row+rows_diff][8]
                        da_bc_list[row][15] = txs_list[1][row+rows_diff][9]
                        da_bc_list[row][16] = txs_list[1][row+rows_diff][10]
                        da_bc_list[row][17] = txs_list[1][row+rows_diff][11]
                        da_bc_list[row][-2] = txs_list[1][row+rows_diff][12]
                    except:
                        pass
                elif da_bc_list[markers][value_idx] == 21:
                    try:
                #     for i_2 in range(row, len(da_bc_list)):
                        da_bc_list[row][7] = txs_list[2][row+rows_diff][1]
                        da_bc_list[row][8] = txs_list[2][row+rows_diff][2]
                        da_bc_list[row][9] = txs_list[2][row+rows_diff][3]
                        da_bc_list[row][10] = txs_list[2][row+rows_diff][4]
                        da_bc_list[row][11] = txs_list[2][row+rows_diff][5]
                        da_bc_list[row][12] = txs_list[2][row+rows_diff][6]
                        da_bc_list[row][13] = txs_list[2][row+rows_diff][7]
                        da_bc_list[row][14] = txs_list[2][row+rows_diff][8]
                        da_bc_list[row][15] = txs_list[2][row+rows_diff][9]
                        da_bc_list[row][16] = txs_list[2][row+rows_diff][10]
                        da_bc_list[row][17] = txs_list[2][row+rows_diff][11]
                        da_bc_list[row][-2] = txs_list[2][row+rows_diff][12]
                    except:
                        pass
                
                elif da_bc_list[markers][value_idx] == 25:
                    try:
                    # for i_3 in range(row, len(da_bc_list)):
                        da_bc_list[row][7] = txs_list[3][row+rows_diff][1]
                        da_bc_list[row][8] = txs_list[3][row+rows_diff][2]
                        da_bc_list[row][9] = txs_list[3][row+rows_diff][3]
                        da_bc_list[row][10] = txs_list[3][row+rows_diff][4]
                        da_bc_list[row][11] = txs_list[3][row+rows_diff][5]
                        da_bc_list[row][12] = txs_list[3][row+rows_diff][6]
                        da_bc_list[row][13] = txs_list[3][row+rows_diff][7]
                        da_bc_list[row][14] = txs_list[3][row+rows_diff][8]
                        da_bc_list[row][15] = txs_list[3][row+rows_diff][9]
                        da_bc_list[row][16] = txs_list[3][row+rows_diff][10]
                        da_bc_list[row][17] = txs_list[3][row+rows_diff][11]
                        da_bc_list[row][-2] = txs_list[3][row+rows_diff][12]
                    except:
                        pass
                    
                elif da_bc_list[markers][value_idx] == 35:
                    try:
                    # for i_4 in range(row, len(da_bc_list)):
                        da_bc_list[row][7] = txs_list[4][row+rows_diff][1]
                        da_bc_list[row][8] = txs_list[4][row+rows_diff][2]
                        da_bc_list[row][9] = txs_list[4][row+rows_diff][3]
                        da_bc_list[row][10] = txs_list[4][row+rows_diff][4]
                        da_bc_list[row][11] = txs_list[4][row+rows_diff][5]
                        da_bc_list[row][12] = txs_list[4][row+rows_diff][6]
                        da_bc_list[row][13] = txs_list[4][row+rows_diff][7]
                        da_bc_list[row][14] = txs_list[4][row+rows_diff][8]
                        da_bc_list[row][15] = txs_list[4][row+rows_diff][9]
                        da_bc_list[row][16] = txs_list[4][row+rows_diff][10]
                        da_bc_list[row][17] = txs_list[4][row+rows_diff][11]
                        da_bc_list[row][-2] = txs_list[4][row+rows_diff][12]
                    except:
                        pass
                    
            #     # if isinstance(element[i][z], str) == True:
            #     #     tx_list[i][z] = element[i][z]
            #     # elif math.isnan(element[i][z]) == False:
            #     #     tx_list[i][z] = element[i][z]



                
df_tx = pd.DataFrame(da_bc_list, columns =da_bs_columns)
# df_tx = merge_vs_tx(start_markers, da_bc_list, txs_list, da_bs_columns)

15
1
905862
905862 855079
50783
25
2
5041386
5041386 4990780
50606
35
3
5580152
5580152 5528458
51694


In [72]:
# da_bc_full = da.merge(df_tx, how='left', left_index=True, right_index=True)

In [73]:
df_tx

,timestamp_x,ECG,DZ,DZDT,Z0,marker,timestamp_y,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,marker_tx,dummy
0,16:56:50.369000,-0.002458,0.068711,-0.007630,35.870574,-9999,16:57:41.545000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
1,16:56:50.370000,-0.004323,0.069047,0.038148,35.870574,-9999,16:57:41.546000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
2,16:56:50.371000,-0.005171,0.068772,-0.099184,35.870574,-9999,16:57:41.547000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
3,16:56:50.372000,-0.005002,0.068833,0.083925,35.870574,-9999,16:57:41.548000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
4,16:56:50.373000,-0.003815,0.068925,0.007630,35.870574,-9999,16:57:41.549000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742104,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742105,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742106,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742107,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# df_tx.to_csv(f'/home/ubuntu/eSportData/jupyter-data//VUAMS_BC_SAMPLE.csv', index=False)

In [77]:
print(df_tx.index[df_tx['marker'] == 35])

Index([5528458, 5528459, 5528460, 5528461, 5528462, 5528463, 5528464, 5528465,
       5528466, 5528467,
       ...
       5588448, 5588449, 5588450, 5588451, 5588452, 5588453, 5588454, 5588455,
       5588456, 5588457],
      dtype='int64', length=60000)


In [80]:
print(df_tx.iloc[5588458])

timestamp_x    18:29:58.827000
ECG                   0.108254
DZ                    0.043534
DZDT                  0.114443
Z0                   35.870574
marker                      36
timestamp_y                NaN
SBP                      110.0
DBP                       90.0
MBP                      100.0
HR                        88.0
SV                        34.8
LVET                     255.0
PI                       685.0
MS                       383.0
CO                         3.0
TPR                      1.966
TPRCGS                  2622.0
marker_tx                  NaN
dummy                      NaN
Name: 5588458, dtype: object


In [81]:
try: 
    # file_path_acc = os.path.join('/home/ubuntu/eSportData/jupyter-data/akcelerometry/S1/S1_299.csv')
    acc = pd.read_csv(file_path_acc, header=None, sep=';', encoding='cp1250', decimal=',', names = ['time', 'wr', 'tl', 'tr'], usecols = ['time', 'wr', 'tl', 'tr'], skiprows = 11)  
    acc = acc.iloc[:-1]
                
    acc['time'] = acc['time'].astype(str)
    acc['time'] = pd.to_datetime(acc['time'])
    
    acc['timestamp'] = acc['time'].apply(lambda x: x + timedelta(microseconds=1000))
    acc = acc.iloc[:-2]
    acc['timestamp'] = pd.to_datetime(acc['timestamp'], format='%H:%M:%S.%f')
    acc = acc.set_index('timestamp')
    acc = acc.resample('ms').ffill()
    acc = acc.reset_index()
    acc['timestamp'] = acc['timestamp'].dt.time
    acc = acc.drop(columns = ['time'])
            #   dropping values of acc to make it equal with txs
    acc_sync_value = df_tx.iloc[0,0]
    acc_sync_idx = acc['timestamp'].index[acc['timestamp'] == acc_sync_value][0]
                # acc = acc.iloc[acc_sync_idx-60000:]
    acc_last_value = df_tx.iloc[-1,0]
    acc_last_idx = acc['timestamp'].index[acc['timestamp'] == acc_last_value][0]
    print(acc_last_value)
                # acc = acc.iloc[acc_sync_idx-60000:]
    acc = acc.iloc[acc_sync_idx: acc_last_idx]
    acc = acc.reset_index(drop=True)
except:
    df_tx['wr'] = None
    df_tx['tl'] = None
    df_tx['tr'] = None


/tmp/ipykernel_2834252/534062649.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc['time'] = pd.to_datetime(acc['time'])


18:32:32.477000


In [82]:
df_tx

,timestamp_x,ECG,DZ,DZDT,Z0,marker,timestamp_y,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,marker_tx,dummy
0,16:56:50.369000,-0.002458,0.068711,-0.007630,35.870574,-9999,16:57:41.545000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
1,16:56:50.370000,-0.004323,0.069047,0.038148,35.870574,-9999,16:57:41.546000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
2,16:56:50.371000,-0.005171,0.068772,-0.099184,35.870574,-9999,16:57:41.547000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
3,16:56:50.372000,-0.005002,0.068833,0.083925,35.870574,-9999,16:57:41.548000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
4,16:56:50.373000,-0.003815,0.068925,0.007630,35.870574,-9999,16:57:41.549000,0.0,0.0,0.0,96.0,0.0,0.0,625.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742104,18:32:32.473000,5.484347,0.989990,0.022889,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742105,18:32:32.474000,5.484347,0.989990,0.068666,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742106,18:32:32.475000,5.484008,0.990112,0.129702,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742107,18:32:32.476000,5.484347,0.990234,0.038148,26.265934,-9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:

df_tx = df_tx.drop(columns = ['timestamp_y'], axis = 1)
df_tx = df_tx.rename(columns={'timestamp_x': 'timestamp'})


In [84]:
df_all = df_tx.merge(acc, how='left', on = 'timestamp')
# df_all = df_tx
df_all['marker_y'] = df_all['marker']
df_all = df_all.drop(columns = ['dummy','marker', 'marker_tx'], axis = 1)
df_all = df_all.rename(columns={'marker_y': 'marker'})
df_all = df_all.iloc[:-2]

In [73]:
# df_all.iloc[:,2] = -9999
# df_all.iloc[:,3] = -9999
# df_all.iloc[:,4] = -9999


In [74]:
df_all

,timestamp,ECG,DZ,DZDT,Z0,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,wr,tl,tr,marker
0,16:54:44.500000,0.023991,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
1,16:54:44.501000,0.023821,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
2,16:54:44.502000,0.023651,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
3,16:54:44.503000,0.025008,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
4,16:54:44.504000,0.025686,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469995,18:09:14.495000,0.038911,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.63,0.0,0.0,-9999
4469996,18:09:14.496000,0.038232,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.63,0.0,0.0,-9999
4469997,18:09:14.497000,0.038402,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.63,0.0,0.0,-9999
4469998,18:09:14.498000,0.038063,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.63,0.0,0.0,-9999


In [77]:
# print(df_all.index[df_all['marker'] == 15])

In [78]:
# print(df_tx.iloc[1598351])

In [85]:
df_all.to_csv(f'/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p245.csv', index=False)

In [ ]:
def extract_time_from_file(filename):
    try:
        with open(filename, 'r') as file:
            # Skip the first row
            next(file)
            # Read the second row
            second_row = next(file).strip()
            # Split by "/"
            parts = second_row.split("/")
            # Return the time part after "/"
            return parts[1]
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return None

# Example usage:
filename = "/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1/160_07060920_DZ.txt"  # Change this to your file path
time_info = extract_time_from_file(filename)
if time_info:
    print("Time information:", time_info)

In [ ]:
start_0 = da_bc.index[da_bc['marker'] ==3298][0]

In [2]:
import pandas as pd

In [86]:
path = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p245.csv'
da = pd.read_csv(path)
da.iloc[:, 2] = -9999
# da = da.iloc[:-300000]
da.iloc[:, 3] = -9999
da.iloc[:, 4] = -9999

In [14]:
da

,timestamp,ECG,DZ,DZDT,Z0,SBP,DBP,MBP,HR,SV,LVET,PI,MS,CO,TPR,TPRCGS,wr,tl,tr,marker
0,16:35:31.026000,-9999.0,-0.276020,0.160220,5.963370,0.0,0.0,0.0,87.0,0.0,0.0,690.0,0.0,0.0,0.0,0.0,46.75,0.0,0.0,-9999
1,16:35:31.027000,-9999.0,-0.275807,0.175479,5.963370,0.0,0.0,0.0,87.0,0.0,0.0,690.0,0.0,0.0,0.0,0.0,46.75,0.0,0.0,-9999
2,16:35:31.028000,-9999.0,-0.275685,0.022889,5.963370,0.0,0.0,0.0,87.0,0.0,0.0,690.0,0.0,0.0,0.0,0.0,46.75,0.0,0.0,-9999
3,16:35:31.029000,-9999.0,-0.275776,-0.221256,5.963370,0.0,0.0,0.0,87.0,0.0,0.0,690.0,0.0,0.0,0.0,0.0,46.75,0.0,0.0,-9999
4,16:35:31.030000,-9999.0,-0.276143,-0.495918,5.963370,0.0,0.0,0.0,87.0,0.0,0.0,690.0,0.0,0.0,0.0,0.0,46.75,0.0,0.0,-9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931465,17:57:42.491000,-9999.0,0.016556,0.053407,11.565324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
4931466,17:57:42.492000,-9999.0,0.016587,0.007630,11.565324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
4931467,17:57:42.493000,-9999.0,0.016556,-0.007630,11.565324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999
4931468,17:57:42.494000,-9999.0,0.016556,-0.007630,11.565324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.0,-9999


In [87]:
da.to_csv(f'/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p245.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
missing_acc = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/s1_output/S1_p136.csv'
df_miss = pd.read_csv(missing_acc)
df_miss